In [1]:
from api_methods.get_followers import get_followers
from model.client import AtProtoClientContextManager
from pymongo import MongoClient
from datetime import datetime

In [2]:
MAX_FOLLOWERS = 1000

MONGO_DB = "bsky"
MONGO_COLLECTION = "cuiaba"

ACTOR_TIME = "did:plc:3z6nl4ujgistw3ija4ig6bgx"

In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client[MONGO_DB]
collection = db[MONGO_COLLECTION]

collection.create_index("user_id", unique=True)

'user_id_1'

In [21]:
def save_to_db(user_dados, seguidores):
    try:
        collection.update_one(
            {"user_id": user_dados['did']},
            {
                "$set": {
                    "display_name": user_dados.get('display_name', 'N/A'),
                    "description": user_dados.get('description', 'N/A'),
                    "handle": user_dados.get('handle', 'N/A'),
                    "followers": list(seguidores),
                    "updated_at": datetime.now()
                },
                "$setOnInsert": {
                    "created_at": datetime.now()
                }
            },
            upsert=True
        )
    except Exception as e:
        print(f"Erro ao salvar dados do usuário {user_dados['did']}: {e}")

In [22]:
with AtProtoClientContextManager() as client:
    actor = ACTOR_TIME
    
    followers_data = [
        {
            "did": follower.did,
            "display_name": follower.display_name or 'N/A',
            "description": follower.description or 'N/A',
            "handle": follower.handle or 'N/A'
        }
        for follower in get_followers(actor, client, MAX_FOLLOWERS)
    ]
    
    print(f"Total de seguidores diretos do ACTOR: {len(followers_data)}")
    
    # Para cada seguidor direto, buscar seus seguidores e salvar no banco
    for follower_data in followers_data:
        follower_id = follower_data['did']
        
        # Recuperar seguidores do seguidor atual
        followers_of_follower = [
            follower.did for follower in get_followers(follower_id, client, MAX_FOLLOWERS)
        ]
        print(f"Total de seguidores do seguidor {follower_id}: {len(followers_of_follower)}")
        
        # Salvar no banco com a lista de seguidores
        save_to_db(follower_data, followers_of_follower)

    print("Coleta concluída!")

Total de seguidores diretos do ACTOR: 53
Total de seguidores do seguidor did:plc:kg5y2va527f2fhopcrmbik6q: 78
Total de seguidores do seguidor did:plc:jvzgwzpadisxldab37m4vbcr: 128
Total de seguidores do seguidor did:plc:pu7adrujg6lsmk533t2dn74r: 111
Total de seguidores do seguidor did:plc:gg3ryl7wp74c6oxcxxewek7v: 112
Total de seguidores do seguidor did:plc:io5ioap2j4qhh7ge7rycyfey: 37
Total de seguidores do seguidor did:plc:toqpgaosgohjwtuwkeo6csta: 5
Total de seguidores do seguidor did:plc:g22u5pmwlvvxfwbavrsm65vb: 38
Total de seguidores do seguidor did:plc:ub7r24bgoctrs6bgdob7iqrg: 63
Total de seguidores do seguidor did:plc:wpqoaeih2mffqxtlx7ff3bb7: 37
Total de seguidores do seguidor did:plc:tfbxy3ft7bftdoxymy752w2c: 24
Total de seguidores do seguidor did:plc:4bjafvjg44jqvoqjfcvjypon: 58
Total de seguidores do seguidor did:plc:4z7oyrrh3wnpsfpfpvjegmp5: 24
Total de seguidores do seguidor did:plc:ecy72n6dz5rvvo7wk6mqif3q: 51
Total de seguidores do seguidor did:plc:xebcfmwblhno4i4qznxf

In [32]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Passo 1: Carregar o arquivo CSV
arquivo_csv = "bsky.cuiaba.csv"  # Substitua pelo nome do seu arquivo
df = pd.read_csv(arquivo_csv)

# Passo 2: Inicializar o grafo direcionado
grafo = nx.DiGraph()

# Passo 3: Criar as conexões do grafo
for _, row in df.iterrows():
    user = row["user_id"]  # Identifica o usuário (dono da linha)
    followers = row.filter(like="followers").dropna()  # Filtra as colunas de seguidores
    
    
    # Adiciona conexões no grafo (seguidores -> usuário)
    for follower in followers:
        grafo.add_edge(follower, user)
        
print(f"Número de nós: {grafo.number_of_nodes()}")
print(f"Número de arestas: {grafo.number_of_edges()}")




Número de nós: 1208
Número de arestas: 3626


In [26]:
nx.write_gexf(grafo, "grafo_seguidores.gexf")
